In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install jieba
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install hanlp
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-28 15:18:58.512415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 15:18:59.628243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/p

In [ ]:
import spacy
from jieba import posseg
from nltk.corpus import wordnet 
import jieba
import nltk
import stanza
import hanlp
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import re
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")
nltk.download('wordnet')
nlp_en = spacy.load("en_core_web_sm")
dep_zh = hanlp.load(hanlp.pretrained.dep.CTB9_DEP_ELECTRA_SMALL)
stanza.download('zh')
nlp_zh = stanza.Pipeline('zh')



[nltk_data] Downloading package wordnet to /root/nltk_data...
100%  47.2 MiB  23.2 MiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/dep/ctb9_dep_electra_small_20220216_100306.zip to /root/.hanlp/dep
100%  26.7 KiB  26.7 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/utils/char_table_20210602_202632.json.zip to /root/.hanlp/utils


Building model ...Downloading https://file.hankcs.com/hanlp/transformers/electra_zh_small_20210706_125427.zip to /root/.hanlp/transformers/electra_zh_small_20210706_125427.zip
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/transformers/electra_zh_small_20210706_125427.zip to /root/.hanlp/transformers


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package   |
----------------------------
| tokenize     | gsdsimp   |
| pos          | gsdsimp   |
| lemma        | gsdsimp   |
| constituency | ctb       |
| depparse     | gsdsimp   |
| sentiment    | ren       |
| ner          | ontonotes |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class Modtrans:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path


    def without_transp(self):  # adj,  verb
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            # en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == 'VERB' and tok_en.text.endswith("ing") and 'go' not in en and 'get' not in en and 'take' not in en]
            en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == 'VERB']
            if en_verb_li != []:
                zh_verb_li = []
                for zh_tok,label in posseg.cut(zh):
                    if label == 'v':
                        zh_verb_li.append(zh_tok)
                if zh_verb_li != []:
                    for zh_verb in zh_verb_li:
                        en_token = tokenizer(zh_verb, return_tensors="pt")
                        generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
                        zh_text_trans = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                        print(zh_text_trans)
                        if zh_text_trans not in en_verb_li:
                            synsets = wordnet.synsets(zh_text_trans)
                            if synsets != []:
                                hypernyms = synsets[0].hypernyms()
                                hyponyms = synsets[0].hyponyms()
                                synonyms = []
                                hyper_li_final = []
                                hypo_li_final = []
                                for synset in synsets:
                                    for lemma in synset.lemmas():
                                        synonyms.append(lemma.name())

                                hyper_li = [h.lemma_names() for h in hypernyms]
                                if hyper_li != []:
                                    hyper_li_final = []
                                    hyper_str = '_'.join(hyper_li[0])
                                    for ph in hyper_li[0]:
                                        if '_' in ph:
                                            new_ph = ph.replace('_',' ')
                                            hyper_li_final.append(new_ph)
                                        else:
                                            hyper_li_final.append(ph)
                                hypo_li = [h.lemma_names() for h in hyponyms]
                                if hypo_li != []:
                                    hypo_li_final = []
                                    for ph in hypo_li[0]:
                                        if '_' in ph:
                                            new_ph = ph.replace('_',' ')
                                            hypo_li_final.append(new_ph)
                                        else:
                                            hypo_li_final.append(ph)
                                

                                if len(list(set(synonyms).intersection(set(en_verb_li)))) == 0 or len(list(set(hyper_li_final).intersection(set(en_verb_li)))) == 0 or len(list(set(hypo_li_final).intersection(set(en_verb_li)))) == 0:

                                      str_align = ''
                                      for en_tok in en_verb_li:

                                          str_en_zh = en_tok+':'+zh_verb+'\n'
                                          str_align += str_en_zh
                                          print(e)
                                          e+=1
                                      print(en)
                                      print(zh)
                                      print(str_align)
                                            
                                      with open(save_path,'a+') as r:
                                          r.write(en+'\n'+zh+'\n'+str_align+'\n')



                    

                
                
                
            

            
            
spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Education/Bi-Education.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature67_mod_mod+transpo/mod_slight_mean_change_verb.txt'             
        
ptcl = Modtrans(spath, save_path)

ptcl.without_transp()




Streaming output truncated to the last 5000 lines.

Yes
130076
130077
130078
This is a climb, note the positive pitch ladder indication on the HUD, the increasing altitude, and decreasing airspeed.
这是一个上升，注意这正俯仰梯度指示，增加高度会降低空速。
note:是
increasing:是
decreasing:是

Rise
130079
130080
130081
This is a climb, note the positive pitch ladder indication on the HUD, the increasing altitude, and decreasing airspeed.
这是一个上升，注意这正俯仰梯度指示，增加高度会降低空速。
note:上升
increasing:上升
decreasing:上升

Notice
130082
130083
130084
This is a climb, note the positive pitch ladder indication on the HUD, the increasing altitude, and decreasing airspeed.
这是一个上升，注意这正俯仰梯度指示，增加高度会降低空速。
note:注意
increasing:注意
decreasing:注意

Ascension
130085
130086
130087
This is a climb, note the positive pitch ladder indication on the HUD, the increasing altitude, and decreasing airspeed.
这是一个上升，注意这正俯仰梯度指示，增加高度会降低空速。
note:俯仰
increasing:俯仰
decreasing:俯仰

Instructions
130088
130089
130090
This is a climb, note the positive pitch ladder indication 

FileNotFoundError: ignored